In [1]:
import json
import boto3
import random
import os
from concurrent.futures import ThreadPoolExecutor
import threading
RUNNING = True
threadLimiter = threading.BoundedSemaphore(6)
resultLock = threading.Lock()

s3 = boto3.resource('s3')

RAM_LIST = list(range(128, 3072 ,64))

In [2]:
def get_task_type_list(workflow_file_path):
    with open(workflow_file_path) as result_json:
        data = json.load(result_json)
    task_type_input_files_map = {}
    for process in data['processes']:
        if process['name'] not in task_type_input_files_map:
            task_type_input_files_map[process['name']] = []
        for in_file in process['ins']:
            if data['signals'][in_file]['name'] not in task_type_input_files_map[process['name']]:
                task_type_input_files_map[process['name']].append(data['signals'][in_file]['name'])

    return task_type_input_files_map

In [3]:
def copy_file_from_to_bucket(copy_source, copy_destination):
    bucket = s3.Bucket('cegielskir')
    bucket.copy(copy_source, copy_destination)

In [4]:
def execute_workflow_task(config, ins, outs, ram, s3_path):
    options = {
        "bucket": "cegielskir",
        "prefix": s3_path
    }
    os.environ['FUNCTION_TYPE'] = str(ram)
    ins_arg = json.dumps(ins)
    outs_arg = json.dumps(outs)
    config_arg = json.dumps(config)
    options_arg = json.dumps(options)
    result = !node awsLambdaCommand.js {"'" + ins_arg + "'"} {"'" + outs_arg + "'"} {"'" + config_arg + "'"} {"'" + options_arg + "'"}
    return ' '.join(result)

In [5]:
def run_single_task(filename, foldername):
    with open(foldername + filename) as single_workflow:
        data = json.load(single_workflow)
    task_name = data['processes'][0]['name']
    process = data['processes'][0]
    config = process['config']
    ins = [ data['signals'][i] for i in process['ins'] ]
    outs = [ data['signals'][i] for i in process['outs'] ]
    run_and_save_results(config, ins, outs, ram, filename.split('_')[0], filename)


In [6]:
def run_and_save_results(config, ins, outs, ram, task_core,task_name):
    print("Starting " + str(task_name) + " with ram " + str(ram) + "   core - " + str(task_core))
    result = execute_workflow_task(config, ins, outs, ram, 'data-collection/montage-0_35/' + task_core)
    save_result(result, './montage-0.35-raw-results-more-data.json', str(ram), task_name)
    threadLimiter.release()


In [7]:
def save_result(result, filename,ram, task_id):
    resultLock.acquire()
    with open(filename) as result_json:
        data = json.load(result_json)
    if ram not in data:
        data[ram] = []
    result_dict = {}
    result_dict['task_id'] = task_id
    result_dict['result'] = str(result)
    data[ram].append(result_dict)
    
    with open(filename, 'w') as result_json:
        data = json.dump(data, result_json, indent=4)
    resultLock.release()

In [8]:
####################### TO BE CHANGED ###########################

exec_folder = 'montageV2_6-compiles'
source_path = 'dc-start-montage-0_35/'
folder_with_workflow_parts = 'montage-0_35/'
workflow_file = './dc-workflow-0.35.json'

###################### END TO BE CHANGED ####################### 

In [36]:
task_in_file_map = get_task_type_list(workflow_file)


for task_type in task_in_file_map:
    for exe_file in s3.Bucket('cegielskir').objects.filter(Prefix=exec_folder):
        copy_exec_source = {
            'Bucket': 'cegielskir',
            'Key': exe_file.key
        }
        
        copy_file_from_to_bucket(copy_exec_source, 'data-collection/' + folder_with_workflow_parts +task_type + '/' + exe_file.key.split('/')[1])
    
#     for in_file in task_in_file_map[task_type]:
#         copy_source = {
#             'Bucket': 'cegielskir',
#             'Key': source_path + in_file
#         }
        
#         copy_file_from_to_bucket(copy_source, 'data-collection/' + folder_with_workflow_parts +task_type + '/' + in_file)
    

In [9]:
files = os.listdir('./' + folder_with_workflow_parts)
#files.reverse()
it = 1
while True:
    print("================================== NEW LOOP ====================================")
    if RUNNING:
        for file in files:
            for ram in RAM_LIST:
                task_core = file.split('_')[0]
                if task_core in ('mConcatFit', 'mBgModel', 'mImgtbl', 'mAdd', 'mViewer'):
                    with open('./' + folder_with_workflow_parts + file) as single_workflow:
                        data = json.load(single_workflow)
                    task_name = data['processes'][0]['name']
                    process = data['processes'][0]
                    config = process['config']
                    ins = [ data['signals'][i] for i in process['ins'] ]
                    outs = [ data['signals'][i] for i in process['outs'] ]
                    threadLimiter.acquire()
                    print("Iteration: " + str(it))
                    threading.Thread(target=run_and_save_results, args=(config, ins, outs, ram, task_core,file)).start()
                    it += 1

================================== NEW LOOP ====================================
Iteration: 1
Starting mBgModel_1.json with ram 128   core - mBgModel
Iteration: 2
Starting mBgModel_1.json with ram 192   core - mBgModel
Iteration: 3
Starting mBgModel_1.json with ram 256   core - mBgModel
Iteration: 4
Starting mBgModel_1.json with ram 320   core - mBgModel
Iteration: 5
Starting mBgModel_1.json with ram 384   core - mBgModel
Iteration: 6
Starting mBgModel_1.json with ram 448   core - mBgModel
Iteration: 7
Starting mBgModel_1.json with ram 512   core - mBgModel
Iteration: 8
Starting mBgModel_1.json with ram 576   core - mBgModel
Iteration: 9
Starting mBgModel_1.json with ram 640   core - mBgModel
Iteration: 10
Starting mBgModel_1.json with ram 704   core - mBgModel
Iteration: 11
Starting mBgModel_1.json with ram 768   core - mBgModel
Iteration: 12
Starting mBgModel_1.json with ram 832   core - mBgModel
Iteration: 13
Starting mBgModel_1.json with ram 896   core - mBgModel
Iteration: 14
Star

Iteration: 121
Starting mBgModel_3.json with ram 1920   core - mBgModel
Iteration: 122
Starting mBgModel_3.json with ram 1984   core - mBgModel
Iteration: 123
Starting mBgModel_3.json with ram 2048   core - mBgModel
Iteration: 124
Starting mBgModel_3.json with ram 2112   core - mBgModel
Iteration: 125
Starting mBgModel_3.json with ram 2176   core - mBgModel
Iteration: 126
Starting mBgModel_3.json with ram 2240   core - mBgModel
Iteration: 127
Starting mBgModel_3.json with ram 2304   core - mBgModel
Iteration: 128
Starting mBgModel_3.json with ram 2368   core - mBgModel
Iteration: 129
Starting mBgModel_3.json with ram 2432   core - mBgModel
Iteration: 130
Starting mBgModel_3.json with ram 2496   core - mBgModel
Iteration: 131
Starting mBgModel_3.json with ram 2560   core - mBgModel
Iteration: 132
Starting mBgModel_3.json with ram 2624   core - mBgModel
Iteration: 133
Starting mBgModel_3.json with ram 2688   core - mBgModel
Iteration: 134
Starting mBgModel_3.json with ram 2752   core - m

Iteration: 239
Starting mViewer_2.json with ram 640   core - mViewer
Iteration: 240
Starting mViewer_2.json with ram 704   core - mViewer
Iteration: 241
Starting mViewer_2.json with ram 768   core - mViewer
Iteration: 242
Starting mViewer_2.json with ram 832   core - mViewer
Iteration: 243
Starting mViewer_2.json with ram 896   core - mViewer
Iteration: 244
Starting mViewer_2.json with ram 960   core - mViewer
Iteration: 245
Starting mViewer_2.json with ram 1024   core - mViewer
Iteration: 246
Starting mViewer_2.json with ram 1088   core - mViewer
Iteration: 247
Starting mViewer_2.json with ram 1152   core - mViewer
Iteration: 248
Starting mViewer_2.json with ram 1216   core - mViewer
Iteration: 249
Starting mViewer_2.json with ram 1280   core - mViewer
Iteration: 250
Starting mViewer_2.json with ram 1344   core - mViewer
Iteration: 251
Starting mViewer_2.json with ram 1408   core - mViewer
Iteration: 252
Starting mViewer_2.json with ram 1472   core - mViewer
Iteration: 253
Starting mV

Iteration: 356
Starting mViewer_1.json with ram 2240   core - mViewer
Iteration: 357
Starting mViewer_1.json with ram 2304   core - mViewer
Iteration: 358
Starting mViewer_1.json with ram 2368   core - mViewer
Iteration: 359
Starting mViewer_1.json with ram 2432   core - mViewer
Iteration: 360
Starting mViewer_1.json with ram 2496   core - mViewer
Iteration: 361
Starting mViewer_1.json with ram 2560   core - mViewer
Iteration: 362
Starting mViewer_1.json with ram 2624   core - mViewer
Iteration: 363
Starting mViewer_1.json with ram 2688   core - mViewer
Iteration: 364
Starting mViewer_1.json with ram 2752   core - mViewer
Iteration: 365
Starting mViewer_1.json with ram 2816   core - mViewer
Iteration: 366
Starting mViewer_1.json with ram 2880   core - mViewer
Iteration: 367
Starting mViewer_1.json with ram 2944   core - mViewer
Iteration: 368
Starting mViewer_1.json with ram 3008   core - mViewer
Iteration: 369
Starting mConcatFit_2.json with ram 128   core - mConcatFit
Iteration: 370


Iteration: 467
Starting mAdd_2.json with ram 512   core - mAdd
Iteration: 468
Starting mAdd_2.json with ram 576   core - mAdd
Iteration: 469
Starting mAdd_2.json with ram 640   core - mAdd
Iteration: 470
Starting mAdd_2.json with ram 704   core - mAdd
Iteration: 471
Starting mAdd_2.json with ram 768   core - mAdd
Iteration: 472
Starting mAdd_2.json with ram 832   core - mAdd
Iteration: 473
Starting mAdd_2.json with ram 896   core - mAdd
Iteration: 474
Starting mAdd_2.json with ram 960   core - mAdd
Iteration: 475
Starting mAdd_2.json with ram 1024   core - mAdd
Iteration: 476
Starting mAdd_2.json with ram 1088   core - mAdd
Iteration: 477
Starting mAdd_2.json with ram 1152   core - mAdd
Iteration: 478
Starting mAdd_2.json with ram 1216   core - mAdd
Iteration: 479
Starting mAdd_2.json with ram 1280   core - mAdd
Iteration: 480
Starting mAdd_2.json with ram 1344   core - mAdd
Iteration: 481
Starting mAdd_2.json with ram 1408   core - mAdd
Iteration: 482
Starting mAdd_2.json with ram 147

...

Iteration: 16570
Starting mConcatFit_2.json with ram 704   core - mConcatFit
Iteration: 16571
Starting mConcatFit_2.json with ram 768   core - mConcatFit
Iteration: 16572
Starting mConcatFit_2.json with ram 832   core - mConcatFit
Iteration: 16573
Starting mConcatFit_2.json with ram 896   core - mConcatFit
Iteration: 16574
Starting mConcatFit_2.json with ram 960   core - mConcatFit
Iteration: 16575
Starting mConcatFit_2.json with ram 1024   core - mConcatFit
Iteration: 16576
Starting mConcatFit_2.json with ram 1088   core - mConcatFit
Iteration: 16577
Starting mConcatFit_2.json with ram 1152   core - mConcatFit
Iteration: 16578
Starting mConcatFit_2.json with ram 1216   core - mConcatFit
Iteration: 16579
Starting mConcatFit_2.json with ram 1280   core - mConcatFit
Iteration: 16580
Starting mConcatFit_2.json with ram 1344   core - mConcatFit
Iteration: 16581
Starting mConcatFit_2.json with ram 1408   core - mConcatFit
Iteration: 16582
Starting mConcatFit_2.json with ram 1472   core - mC

Iteration: 16680
Starting mAdd_2.json with ram 1856   core - mAdd
Iteration: 16681
Starting mAdd_2.json with ram 1920   core - mAdd
Iteration: 16682
Starting mAdd_2.json with ram 1984   core - mAdd
Iteration: 16683
Starting mAdd_2.json with ram 2048   core - mAdd
Iteration: 16684
Starting mAdd_2.json with ram 2112   core - mAdd
Iteration: 16685
Starting mAdd_2.json with ram 2176   core - mAdd
Iteration: 16686
Starting mAdd_2.json with ram 2240   core - mAdd
Iteration: 16687
Starting mAdd_2.json with ram 2304   core - mAdd
Iteration: 16688
Starting mAdd_2.json with ram 2368   core - mAdd
Iteration: 16689
Starting mAdd_2.json with ram 2432   core - mAdd
Iteration: 16690
Starting mAdd_2.json with ram 2496   core - mAdd
Iteration: 16691
Starting mAdd_2.json with ram 2560   core - mAdd
Iteration: 16692
Starting mAdd_2.json with ram 2624   core - mAdd
Iteration: 16693
Starting mAdd_2.json with ram 2688   core - mAdd
Iteration: 16694
Starting mAdd_2.json with ram 2752   core - mAdd
Iteration:

Iteration: 16796
Starting mConcatFit_3.json with ram 448   core - mConcatFit
Iteration: 16797
Starting mConcatFit_3.json with ram 512   core - mConcatFit
Iteration: 16798
Starting mConcatFit_3.json with ram 576   core - mConcatFit
Iteration: 16799
Starting mConcatFit_3.json with ram 640   core - mConcatFit
Iteration: 16800
Starting mConcatFit_3.json with ram 704   core - mConcatFit
Iteration: 16801
Starting mConcatFit_3.json with ram 768   core - mConcatFit
Iteration: 16802
Starting mConcatFit_3.json with ram 832   core - mConcatFit
Iteration: 16803
Starting mConcatFit_3.json with ram 896   core - mConcatFit
Iteration: 16804
Starting mConcatFit_3.json with ram 960   core - mConcatFit
Iteration: 16805
Starting mConcatFit_3.json with ram 1024   core - mConcatFit
Iteration: 16806
Starting mConcatFit_3.json with ram 1088   core - mConcatFit
Iteration: 16807
Starting mConcatFit_3.json with ram 1152   core - mConcatFit
Iteration: 16808
Starting mConcatFit_3.json with ram 1216   core - mConca

Iteration: 16910
Starting mAdd_1.json with ram 1856   core - mAdd
Iteration: 16911
Starting mAdd_1.json with ram 1920   core - mAdd
Iteration: 16912
Starting mAdd_1.json with ram 1984   core - mAdd
Iteration: 16913
Starting mAdd_1.json with ram 2048   core - mAdd
Iteration: 16914
Starting mAdd_1.json with ram 2112   core - mAdd
Iteration: 16915
Starting mAdd_1.json with ram 2176   core - mAdd
Iteration: 16916
Starting mAdd_1.json with ram 2240   core - mAdd
Iteration: 16917
Starting mAdd_1.json with ram 2304   core - mAdd
Iteration: 16918
Starting mAdd_1.json with ram 2368   core - mAdd
Iteration: 16919
Starting mAdd_1.json with ram 2432   core - mAdd
Iteration: 16920
Starting mAdd_1.json with ram 2496   core - mAdd
Iteration: 16921
Starting mAdd_1.json with ram 2560   core - mAdd
Iteration: 16922
Starting mAdd_1.json with ram 2624   core - mAdd
Iteration: 16923
Starting mAdd_1.json with ram 2688   core - mAdd
Iteration: 16924
Starting mAdd_1.json with ram 2752   core - mAdd
Iteration:

KeyboardInterrupt: 